# Constraint-based sampling for metabolic flux analysis (MFA)

Here we jump into the (for now) final step in our metabolic flux analysis (MFA) pipeline. After constructing an INCA script, running it in MATLAB and reimporting the data we're now here. This example notebook will guide you through different ways to integrate your MFA results into COBRA models and how to make them more reliable.

In [1]:
import pandas as pd
import numpy as np
import time
import ast
import sys
import escher
import cobra
from gurobipy import Model as GRBModel
import re
from BFAIR.INCA import INCA_reimport
from BFAIR.INCA.sampling import *

Academic license - for non-commercial use only - expires 2021-05-28
Using license file /Users/matmat/gurobi.lic


#### INCA re-import

First, let's reimport the data using our `BFAIR INCA_reimport` tools

The data is taken from a different folder

In [2]:
%cd /Users/matmat/Documents/Random data/Stefano
filename = 'data/MFA_modelInputsData/TestFile.mat'

/Users/matmat/Documents/Random data/Stefano


Then we move back to our example folder to get the remaining input

In [3]:
%cd /Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples
#filename = 'data/MFA_modelInputsData/TestFile.mat'
simulation_info = pd.read_csv('data/MFA_modelInputsData/Re-import/experimentalMS_data_I.csv')
simulation_id = 'WTEColi_113C80_U13C20_01'

/Users/matmat/Documents/GitHub/AutoFlow-OmicsDataHandling/examples


Here we re-import the INCA output

In [4]:
reimport_data = INCA_reimport()
(fittedData,
 fittedFluxes,
 fittedFragments,
 fittedMeasuredFluxes,
 fittedMeasuredFragments,
 fittedMeasuredFluxResiduals,
 fittedMeasuredFragmentResiduals,
 simulationParameters) = reimport_data.reimport(
    filename,
    simulation_info,
    simulation_id
)

Here we import the model

In [5]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [6]:
original_solution = model.optimize()
original_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000e+00
EX_cmp_e,0.000000,-2.965572e-01
EX_co2_e,19.675223,0.000000e+00
EX_cobalt2_e,-0.000025,-0.000000e+00
DM_4crsol_c,0.000219,0.000000e+00
...,...,...
RNDR4,0.000000,-2.073827e-03
RNDR4b,0.000000,-2.073827e-03
RNTR1c2,0.025705,-8.673617e-18
RNTR2c2,0.026541,-8.673617e-18


Let's see what happens now when we add our new bound constraints

In [7]:
model = add_constraints(model, fittedFluxes)
model.optimize()

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpgem77a3n.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp13v4u2ue.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


<Solution infeasible at 0x7f7fe367d850>

Oh... That's not good... Well that sucks, seems like we have to deal with an infeasible solution. There are two straight forward ways: exclusion and relaxation

#### Dealing with infeasible solutions - exclusion

The easier way to deal with this issue is to simply exclude the constraints that render a model infeasible. We can do that by adding the calculated bounds one by one. If we come across a reaction whose bounds cause trouble, we restart the process and skip this one. This might have to be done a few times to exclude all troublemakers. the `add_feasible_constraints()` functions takes care of that for us. Let's reset the model first.

In [8]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [9]:
model, problems = add_feasible_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp9xmm58c6.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7fzumdn_.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpnica04ld.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp7rk72v9f.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding RPI
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpeic6c4is.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmprhz0q1w2

The `model` is our newly constrained model and the problematic reactions can be listed in `problems`.

In [10]:
model

Name,iJO1366
Memory address,0x07f7fcaf9e520
Number of metabolites,1805
Number of reactions,2583
Number of groups,0
Objective expression,1.0*BIOMASS_Ec_iJO1366_core_53p95M - 1.0*BIOMASS_Ec_iJO1366_core_53p95M_reverse_5c8b1
Compartments,"cytosol, extracellular space, periplasm"


In [11]:
problems

['ASPTA', 'RPI']

Now let's see what an effect these new bounds had on the predicted growth rate (the objective value) of our model

In [12]:
new_bounds_solution = model.optimize()
new_bounds_solution

,fluxes,reduced_costs
EX_cm_e,0.000000,0.000000
EX_cmp_e,0.000000,-1.104597
EX_co2_e,14.276815,0.000000
EX_cobalt2_e,-0.000007,-0.000000
DM_4crsol_c,0.000062,0.000000
...,...,...
RNDR4,0.000000,0.000000
RNDR4b,0.000000,0.000000
RNTR1c2,0.000000,0.000000
RNTR2c2,0.007461,0.000000


And here's the star of the show, our sampling method. We trust our models because... we have to! And because smart people that knew what they were doing set them up. So in order to gain more confidence in our MFA data, we sample the model after adding the calculated bound for some of the reactions and re-calculate the fluxes a number of time. Then, we take the mean and take that as the most trustworthy calculated flux. These fluxes can be visualized, for example in tools like `Escher`

In [13]:
sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp6722r96l.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpt6bs7xvf.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [14]:
sampled_fluxes

,EX_cm_e,EX_cmp_e,EX_co2_e,EX_cobalt2_e,DM_4crsol_c,DM_5drib_c,DM_aacald_c,DM_amob_c,DM_mththf_c,EX_colipa_e,...,UREAtex,RNDR2b,UREAtpp,RNDR3,RNDR3b,RNDR4,RNDR4b,RNTR1c2,RNTR2c2,RNTR3c2
0,0.0,0.0,33.609530,-5.193709e-11,4.803987e-10,1.036251e-07,0.0,4.308275e-12,0.000001,6.780951e-08,...,-0.000003,0.000006,-0.000003,5.716573e-08,1.447784e-09,0.000460,5.357297e-08,0.000031,5.238063e-07,1.236983e-08
1,0.0,0.0,33.609566,-5.323893e-11,4.924829e-10,6.620677e-08,0.0,4.416653e-12,0.000002,5.632315e-08,...,-0.000002,0.000006,-0.000002,5.824845e-08,1.580191e-09,0.000460,5.356920e-08,0.000031,3.437153e-07,6.424933e-09
2,0.0,0.0,33.606182,-7.134094e-10,6.621539e-09,2.905226e-08,0.0,5.938577e-11,0.000003,3.570224e-08,...,-0.000003,0.000006,-0.000003,7.632223e-07,1.300203e-09,0.000462,5.356281e-08,0.000031,2.885633e-07,6.424103e-09
3,0.0,0.0,33.605917,-7.161766e-10,6.644565e-09,2.886066e-08,0.0,5.959227e-11,0.000009,3.228566e-08,...,-0.000002,0.000006,-0.000002,7.628048e-07,4.767558e-09,0.000462,5.355753e-08,0.000031,1.049949e-07,6.423412e-09
4,0.0,0.0,33.597916,-2.197891e-09,2.040992e-08,4.332918e-05,0.0,1.830484e-10,0.000208,4.810555e-08,...,-0.000016,0.000007,-0.000016,8.594006e-07,1.488190e-06,0.000467,1.490238e-06,0.000033,6.078165e-05,7.723116e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,0.0,0.0,28.903821,-4.105658e-07,3.725811e-06,2.658014e-03,0.0,3.341260e-08,0.004557,5.196937e-04,...,-0.020565,0.066800,-0.020565,1.210649e-03,4.018250e-04,0.003219,2.768184e-02,0.110403,3.820177e-02,2.279911e-03
96,0.0,0.0,28.933999,-3.905764e-07,3.546485e-06,5.287049e-03,0.0,3.180424e-08,0.006778,5.333501e-04,...,-0.021303,0.066849,-0.021303,1.241302e-03,3.524871e-04,0.010638,2.770985e-02,0.116658,3.823790e-02,2.283920e-03
97,0.0,0.0,28.877574,-3.784858e-07,3.434467e-06,3.536589e-03,0.0,3.079970e-08,0.005708,5.381539e-04,...,-0.021220,0.066659,-0.021220,1.422424e-03,3.490089e-04,0.010555,2.764085e-02,0.115581,3.718307e-02,2.276375e-03
98,0.0,0.0,28.788300,-3.070992e-07,2.769257e-06,3.537453e-03,0.0,2.468476e-08,0.003437,6.008855e-04,...,-0.021236,0.066691,-0.021236,1.359105e-03,3.436077e-04,0.010493,2.765046e-02,0.087748,3.405403e-02,2.273398e-03


In [15]:
fluxes_sampling = reshape_fluxes_escher(sampled_fluxes)

In [16]:
sampled_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = fluxes_sampling).display_in_notebook()
sampled_flux_map

There are also other ways to cosolidate the MFA calculations and the constraint based flux predictions. One of these is lMOMA (linear Minimization Of Metabolic Adjustment). MOMA assumes that the fluxes before and after adding the new constraints should be similar, so it aims to predicting an optimum for the newly constrined model while keeping the differences to the original model small. We suggest using pFBA (parsimonious Flux Balance Analysis) instead of regular FBA for this step as pFBA aims to keep the overal fluxes low.

In [17]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')
model, problems = add_feasible_constraints(model, fittedFluxes)
pfba_solution = cobra.flux_analysis.pfba(model)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp3e0xgf6d.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpfvpzp72b.lp
Reading time = 0.03 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding ASPTA
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpubun4hg9.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


/Users/matmat/opt/anaconda3/envs/bfair/lib/python3.8/site-packages/cobra/util/solver.py:430: UserWarning: solver status is 'infeasible'
  warn("solver status is '{}'".format(status), UserWarning)


Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpwrc0g2k9.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Solution infeasible if adding RPI
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpdrj2y8kv.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp71cs44of.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
---------------------------
Total number of restarts: 2
add_feasible_constraints takes 0h: 0min: 45sec to run
--- end ---


In [18]:
moma_after_MFA = cobra.flux_analysis.moma(model=model, solution=pfba_solution, linear=True)

For some reason I get weird values when I just look at the objective value here, both for pFBA and for lMOMA, so I just look at the biomass function instead. This should produce a similar result as the regular FBA prediction after adding the new constraints

In [19]:
moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

0.27614926420028335

In [20]:
print(f'Before adding new constraints:             {original_solution.objective_value:.2f}')
print(f'After adding new constraints:              {new_bounds_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, objV: {pfba_solution.objective_value:.2f}')
print(f'After adding new constraints + pFBA, BM:   {pfba_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')
print(f'After adding new constraints + MOMA, objV: {moma_after_MFA.objective_value:.2f}')
print(f'After adding new constraints + MOMA, BM:   {moma_after_MFA.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]:.2f}')

Before adding new constraints:             0.98
After adding new constraints:              0.28
After adding new constraints + pFBA, objV: 370.91
After adding new constraints + pFBA, BM:   0.28
After adding new constraints + MOMA, objV: 0.00
After adding new constraints + MOMA, BM:   0.28


Of course also the MOMA results can be visualized with `Escher`. The `reshape_fluxes_escher()` can take both pandas DataFrames or cobra solutions as an input.

In [21]:
fluxes_moma = reshape_fluxes_escher(moma_after_MFA)

In [22]:
moma_flux_map = escher.Builder('e_coli_core.Core metabolism',
                               reaction_data = fluxes_moma).display_in_notebook()
moma_flux_map

#### Dealing with infeasible solutions - relaxation

Another way of dealing with infeasible models is to relax the added constraints to the point that it works again. You will need to have the Gurobi solver installed for this. The same principle is used in the `BFAIR thermo` tools. For that we add our constraints to a model that will now be infeasible.

In [23]:
model = cobra.io.load_json_model('data/FIA_MS_example/database_files/iJO1366.json')

In [24]:
model = add_constraints(model, fittedFluxes)

--- start ---
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp4wm_zb15.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmprceeemse.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
add_constraints takes 0h: 0min: 10sec to run
--- end ---


In [25]:
model.optimize()

<Solution infeasible at 0x7f7f7cd4d4f0>

Then we make use of the handy `bound_relaxation()` function that will test our model to figure out which of these added bounds need to be adjusted and return a DataFrame that describes the affected functions and the gravity of the suggested changes. If we allow this function to be `desctructive` it will adjust the input model right away.

In [26]:
bound_relaxation(model, fittedFluxes, destructive=True)

--- start ---
bound_relaxation takes 0h: 0min: 0sec to run
--- end ---


,bound_change,subsystem
reaction,,
ASPTA,-1.3,Alanine and Aspartate Metabolism
DAPDC,-0.2,Threonine and Lysine Metabolism


Let's see if our model is feasible now.

In [27]:
relaxed_solution = model.optimize()
relaxed_solution

,fluxes,reduced_costs
EX_cm_e,0.000000e+00,0.0
EX_cmp_e,0.000000e+00,0.0
EX_co2_e,1.125075e+01,0.0
EX_cobalt2_e,-3.213772e-12,-0.0
DM_4crsol_c,7.849232e-09,0.0
...,...,...
RNDR4,0.000000e+00,0.0
RNDR4b,0.000000e+00,0.0
RNTR1c2,0.000000e+00,0.0
RNTR2c2,0.000000e+00,0.0


In [28]:
relaxed_solution.fluxes["BIOMASS_Ec_iJO1366_core_53p95M"]

1.285508972317679e-07

And now we're also at the point where we can let the power of constraint based models work for us and make use of the tools mentioned above in order to adjust the fluxes calculated using MFA so that they nicely fit into our model.

In [29]:
relaxed_sampled_fluxes = cobra.sampling.sample(model, n=100, processes=2)

Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmp3c8zg5v1.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros
Read LP format model from file /var/folders/mb/7cs2dcbn369_w97fqkfx47brjcxl49/T/tmpaq8srze4.lp
Reading time = 0.02 seconds
: 1805 rows, 5166 columns, 20366 nonzeros


In [30]:
relaxed_fluxes_sampling = reshape_fluxes_escher(relaxed_sampled_fluxes)

In [31]:
relaxed_flux_map = escher.Builder('e_coli_core.Core metabolism',
                                  reaction_data = relaxed_fluxes_sampling).display_in_notebook()
relaxed_flux_map